## Aneks preprocessing

In [3]:
import json
import pandas as pd
from torch.utils.data import random_split
from transformers import AutoTokenizer


at = AutoTokenizer.from_pretrained('../data/tokens/')
aneks = pd.read_csv('../data/aneks/raw/anekdotov_net.csv')
aneks2 = pd.read_csv('../data/aneks/raw/anekdotov_net2.csv')
rofls = pd.read_csv('../data/aneks/raw/anekdotov_net_microrofls.csv')
total = pd.concat([aneks, aneks2, rofls], axis=0)

total = total.drop_duplicates('anek')
total = total['anek']

In [4]:
tokenized = total.apply(lambda x: at.tokenize(x))
total = total[tokenized.apply(lambda x: len(x)) < 512]
total.reset_index(drop=True, inplace=True)
train, test = random_split(total, (int(len(total) * 0.9), len(total) - int(len(total) * 0.9)))

In [5]:
for name, df in zip(['train', 'test'], [train, test]):
    anek_dict = {}
    for i, value in enumerate(df):
        anek_dict[i] = value

    with open(f'../data/aneks/processed/{name}.json', 'w') as fp:
        json.dump(anek_dict, fp)

## Vocab preprocessing

In [6]:
import json

tokens = {}
embeddings = {}
with open('../data/vocab/raw/bpe.txt', 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f):
        token, values = line.split(' ', maxsplit=1)
        tokens[token] = idx
        embeddings[idx] = list(map(float,values.split()))

with open('../data/vocab/processed/word2idx.json', 'w') as fp:
    json.dump(tokens, fp)

with open('../data/vocab/processed/idx2embedding.json', 'w') as fp:
    json.dump(embeddings, fp)